# Import

In [17]:
# #.py modules
# import acquire as ac
# import prepare as p
# import explore as ex
# import evaluate as ev
import wrangle as w

#standard
import pandas as pd
import numpy as np
import re

#scraping
import requests
from requests import get
from bs4 import BeautifulSoup

#file
import os
import json
from google.oauth2 import service_account
import pandas_gbq

#vizz
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#preprocess
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#split and model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#set random state
random_state=123

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [18]:
# credentials = service_account.Credentials.from_service_account_file(
#     'my-ds-projects-d864a770b51b.json',
# )

In [19]:
# df = pandas_gbq.read_gbq("select * from cfpb_complaints.complaint_database",dialect='standard',project_id="my-ds-projects",use_bqstorage_api=True,credentials=credentials)
# df

In [20]:
# df.to_csv('ccr.csv')

In [21]:
# df = pd.read_csv('ccr.csv')

In [22]:
df = w.check_file_exists_gbq('cfpb.csv', 'my-ds-projects-d864a770b51b.json')

csv file found and loaded


In [23]:
df.head()

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2022-07-16,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19136,NaN,Consent not provided,Web,2022-07-16,Closed with explanation,True,NaN,5777486
1,2023-01-23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NV,89139.0,NaN,Consent not provided,Web,2023-01-23,Closed with explanation,True,NaN,6476526
2,2022-06-15,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NC,27591.0,NaN,Consent not provided,Web,2022-06-15,Closed with explanation,True,NaN,5675690
3,2022-10-09,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,OH,44266.0,NaN,Consent not provided,Web,2022-10-09,Closed with non-monetary relief,True,NaN,6064894
4,2019-03-02,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information is missing that should be on the r...,"I was locked out of my credit report by XXXX, ...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,IL,62702.0,NaN,Consent provided,Web,2019-03-02,Closed with explanation,True,NaN,3167321


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3458906 entries, 0 to 3458905
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   date_received                 object
 1   product                       object
 2   subproduct                    object
 3   issue                         object
 4   subissue                      object
 5   consumer_complaint_narrative  object
 6   company_public_response       object
 7   company_name                  object
 8   state                         object
 9   zip_code                      object
 10  tags                          object
 11  consumer_consent_provided     object
 12  submitted_via                 object
 13  date_sent_to_company          object
 14  company_response_to_consumer  object
 15  timely_response               bool  
 16  consumer_disputed             object
 17  complaint_id                  int64 
dtypes: bool(1), int64(1), object(16)
memory us

In [25]:
# df.describe()

## data response is the target

In [26]:
# p.basic_clean(df)

In [27]:
# import os
# from google.cloud import bigquery

# # Set the environment variable
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'my-ds-projects-d864a770b51b.json'

# # Initialize a BigQuery client
# client = bigquery.Client()

# # Now you can use the client...


In [28]:
# from google.cloud import bigquery

# # Initialize a BigQuery client
# client = bigquery.Client()

# # Define your query
# query = """
#     SELECT *
#     FROM `bigquery-public-data.cfpb_complaints.complaint_database`
# """

# # Run the query
# query_job = client.query(query)

# # Get the results as a pandas DataFrame
# df_2 = query_job.to_dataframe()

# # Now you can analyze the data using pandas, for example:
# summary = df_2.describe(include='all')


In [29]:
# summary

In [30]:
# df_2.to_csv('cfpb.csv')

In [31]:
# Print the first few rows of the DataFrame
# df_2.head()

In [32]:
# Print the summary statistics of the DataFrame
# print(df_2.describe())

In [33]:
# Print the column names
# print(df_2.columns)

In [34]:
# Count the number of non-NA values
# print(df_2.count())

In [36]:
# Compute pairwise correlation of columns
print(df.corr())

                 timely_response  complaint_id
timely_response         1.000000      0.062366
complaint_id            0.062366      1.000000


In [37]:
# df.dropna()

In [38]:
import nltk

# nltk.download('vader_lexicon')

In [39]:
# df.info()

In [40]:
# df.product()

In [41]:
# df.head()

In [42]:
# w.data_summary(df)

In [43]:
# df_2['product'].value_counts()

In [44]:
# df_2['subproduct'].value_counts()

In [45]:
# df_2['issue'].value_counts()

In [46]:
# df_2['company_name'].value_counts()

In [47]:
# df_2['state'].value_counts()

In [48]:
# df_2['zip_code'].value_counts()

In [49]:
# df_2['tags'].value_counts()

In [50]:
# df_2['consumer_consent_provided'].value_counts()

In [51]:
# df_2['submitted_via'].value_counts()

In [52]:
# df_2['company_response_to_consumer'].value_counts()

In [53]:
# df_2['timely_response'].value_counts()

In [54]:
# df_2['consumer_disputed'].value_counts()

In [55]:
# df_2['company_public_response'].value_counts()

In [56]:
df = w.clean_data(df)

In [57]:
df.head()

,date_received,product,narrative,company_name,state,tags,company_response_to_consumer
0,2022-07-16,"Credit reporting, credit repair services, or o...",Improper use of your report Reporting company ...,Experian Information Solutions Inc.,PA,Average Person,Closed with explanation
1,2023-01-23,"Credit reporting, credit repair services, or o...",Improper use of your report Reporting company ...,Experian Information Solutions Inc.,NV,Average Person,Closed with explanation
2,2022-06-15,"Credit reporting, credit repair services, or o...",Improper use of your report Reporting company ...,Experian Information Solutions Inc.,NC,Average Person,Closed with explanation
3,2022-10-09,"Credit reporting, credit repair services, or o...",Improper use of your report Reporting company ...,Experian Information Solutions Inc.,OH,Average Person,Closed with non-monetary relief
4,2019-03-02,"Credit reporting, credit repair services, or o...",Incorrect information on your report Informati...,Experian Information Solutions Inc.,IL,Average Person,Closed with explanation


In [58]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# # Replace None with '' (empty string)
# df_2['narrative'] = df_2['narrative'].fillna('')

# Apply the sentiment intensity analyzer to the 'narrative' column
df['sentiment'] = df['narrative'].apply(lambda complaint: sia.polarity_scores(complaint))

# Now each complaint has a sentiment score


In [59]:
df['sentiment']

0          {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1          {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
2          {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
3          {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4          {'neg': 0.113, 'neu': 0.828, 'pos': 0.06, 'com...
                                 ...                        
3458901    {'neg': 0.0, 'neu': 0.935, 'pos': 0.065, 'comp...
3458902    {'neg': 0.184, 'neu': 0.661, 'pos': 0.155, 'co...
3458903    {'neg': 0.191, 'neu': 0.659, 'pos': 0.15, 'com...
3458904    {'neg': 0.092, 'neu': 0.753, 'pos': 0.155, 'co...
3458905    {'neg': 0.145, 'neu': 0.785, 'pos': 0.07, 'com...
Name: sentiment, Length: 3355342, dtype: object